In [1]:
import sys
sys.path.append("/home/adityasv/COILv2/SparseRetrieval/COIL")

import json
import torch
import os
from torch import nn
from pprint import pprint
from collections import defaultdict

from tqdm.auto import tqdm
from beir.datasets.data_loader import GenericDataLoader
from transformers import PreTrainedModel
from transformers import AutoModelForMaskedLM, AutoConfig, AutoTokenizer
from transformers.modeling_outputs import  MaskedLMOutput
from arguments import ModelArguments

from torch import Tensor
from typing import Dict, List, Tuple, Iterable
from torch.cuda.amp import autocast

from modeling import COILWithExpansion
print('done')

2022-04-13 19:38:58.950629: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /bos/usr0/adityasv/anaconda3/lib/
2022-04-13 19:38:58.950664: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


done


In [2]:
# %%
model_path = "/bos/tmp15/adityasv/COILv2-vkeshava/model_coil_with_expansion_splade_flops_q0.0008_d0.0006_expansion_weight1_tokens_weight1_expansion-norm_max_expansion_weight_not_learned_init_distilbert_base/"

data_args, model_args, train_args = torch.load(os.path.join(model_path, "args.pt"))

config = AutoConfig.from_pretrained(
        model_path,
        num_labels=1,
        cache_dir=model_args.cache_dir,
    )

model = COILWithExpansion.from_pretrained(model_args, data_args, train_args, model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

id2tok = {v:k for k,v in tokenizer.vocab.items()}

if torch.cuda.is_available():
    model.to('cuda')
    model.eval()



In [3]:
data_path = "/bos/tmp15/adityasv/DomainAdaptation/MSMARCO/msmarco"
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="dev")

In [4]:
def expansion_activation(x, attention_mask):
    if self.model_args.expansion_normalization == "max":
        cloned_attention_mask = attention_mask.clone()
        activation = torch.log(1 + torch.relu(x))
        activation_masked = activation * cloned_attention_mask.unsqueeze(-1)
        expansion_scores = torch.max(activation_masked, dim=1).values
        return expansion_scores, activation_masked
    
def get_scores(query, doc):
    doc = [doc.get("title") + " " + doc.get("text")]
    query = [query]
    doc = tokenizer(doc, return_tensors='pt')
    query = tokenizer(query, return_tensors='pt')

    if torch.cuda.is_available():
        doc = {k: v.cuda() for k,v in doc.items()}
        query = {k: v.cuda() for k,v in query.items()}

    with torch.no_grad():
        scores, tok_scores, expansion_scores, qry_expansion, doc_expansion, qry_expansion_indices, doc_expansion_indices = model(query, doc, return_activations=True)
    qry_expansion_indices.squeeze_()
    doc_expansion_indices.squeeze_()
    
    qry_expansion_reverse_map = defaultdict(list)
    doc_expansion_reverse_map = defaultdict(list)
    
    qry_expansion = torch.nonzero(qry_expansion)[:,1].cpu().numpy().tolist()
    doc_expansion = torch.nonzero(doc_expansion)[:,1].cpu().numpy().tolist()
    
    for i in qry_expansion:
        j = qry_expansion_indices[i].item()
        qry_expansion_reverse_map[id2tok[query['input_ids'][0][j].item()]].append(id2tok[i])
    
    for i in doc_expansion:
        j = doc_expansion_indices[i].item()
        doc_expansion_reverse_map[id2tok[doc['input_ids'][0][j].item()]].append(id2tok[i])

    q_doc_intersection = set(id2tok[i] for i in qry_expansion).intersection(set(id2tok[i] for i in doc_expansion))
        
    return scores, tok_scores, expansion_scores, "\n".join([id2tok[i] for i in qry_expansion]), "\n".join([id2tok[i] for i in doc_expansion]) , qry_expansion_reverse_map, doc_expansion_reverse_map, q_doc_intersection


In [6]:
# query='who needs dialysis'
# doc = {"text": "A dogma has haunted the study and treatment of female infertility for more than half a century. It states that a baby girl is born with an ever-diminishing number of egg cells which cannot be renewed or replenished during her life, and that when she runs out of these eggs an irreversible menopause begins.", "title": ""}

query = queries['421813']
doc = corpus['2554945']

tok_doc = doc.get("title") + " " + doc.get("text")
tok_query = query

tok_query = tokenizer.tokenize(tok_query)
tok_doc = tokenizer.tokenize(tok_doc)

print(query)
print(doc)
print()


print(tok_query)
print(tok_doc)
print("*"*50)
print(set(tok_query).intersection(set(tok_doc)))
print("*"*50)

scores, tok_scores, expansion_scores, qry_expansion, doc_expansion, qry_expansion_reverse_map, doc_expansion_reverse_map, q_doc_intersection  = get_scores(query, doc)

print("scores", scores)
print("tok_scores", tok_scores)
print("expansion_scores", expansion_scores)
print("*"*50)
print(q_doc_intersection)
print("*"*50)
pprint(qry_expansion_reverse_map)
print("*"*50)
pprint(doc_expansion_reverse_map)
print("*"*50)


is protonix an antacid
{'text': 'BRAND NAME: Protonix. DRUG CLASS AND MECHANISM: Pantoprazole is in a class of drugs called proton pump inhibitors (PPIs), which block the production of acid by the stomach. Other drugs in the same class include lansoprazole (Prevacid), omeprazole (Prilosec) and rabeprazole (Aciphex). Proton pump inhibitors are used for the treatment of conditions such as ulcers, gastroesophageal reflux disease (GERD) and Zollinger-Ellison syndrome that are caused by stomach acid.', 'title': ''}

['is', 'proton', '##ix', 'an', 'ant', '##ac', '##id']
['brand', 'name', ':', 'proton', '##ix', '.', 'drug', 'class', 'and', 'mechanism', ':', 'pan', '##top', '##raz', '##ole', 'is', 'in', 'a', 'class', 'of', 'drugs', 'called', 'proton', 'pump', 'inhibitors', '(', 'pp', '##is', ')', ',', 'which', 'block', 'the', 'production', 'of', 'acid', 'by', 'the', 'stomach', '.', 'other', 'drugs', 'in', 'the', 'same', 'class', 'include', 'lan', '##sop', '##raz', '##ole', '(', 'pre', '##vac',